<a href="https://colab.research.google.com/github/DSalux/School_project/blob/main/YAP_0_92_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sentence-transformers

In [2]:
pip install accelerate -U

In [3]:
pip install pyarrow==12.0.1 datasets

In [4]:
pip install transformers[torch]

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sentence_transformers import (SentenceTransformer, InputExample, util, losses)
from torch.utils.data import DataLoader
from datasets import Dataset
import torch
import transformers
import accelerate

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
data = pd.read_csv('/content/sample_data/Школы.csv')

In [7]:
data

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург
...,...,...,...
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край


In [8]:
data['name'] = data['name'] + ' ' + data['region']

In [9]:
keyboard_mapping = {
    'а': 'впк',
    'б': 'юьт',
    'в': 'фыа',
    'г': 'нрош',
    'д': 'жзлб',
    'е': 'капрнг',
    'ё': 'эъхж',
    'ж': 'эщзб',
    'з': 'хщджэ',
    'и': 'ншкг',
    'й': 'цук',
    'к': 'нш',
    'л': 'шд',
    'м': 'ить',
    'н': 'шт',
    'о': 'аеи',
    'п': 'аев',
    'р': 'нкт',
    'с': 'миич',
    'т': 'еь',
    'у': 'гек',
    'ф': 'сро',
    'х': 'ясывф',
    'ц': 'му',
    'ч': 'сь',
    'ш': 'нрм',
    'щ': 'лт',
    'ъ': 'и',
    'ы': 'ой',
    'ь': 'сми',
    'э': 'дз',
    'ю': 'бл',
    'я': 'чс'
}

In [10]:
def generate_errors(name, num_errors=5):
    errors = []
    for _ in range(num_errors):
        error = list(name)
        num_changes = random.randint(1, 5)  # Количество изменений в слове
        for _ in range(num_changes):
            index = random.randint(0, len(name) - 1)
            if name[index] in keyboard_mapping:
                possible_replacements = keyboard_mapping[name[index]]
                error[index] = random.choice(possible_replacements)
        errors.append(''.join(error))
    return errors

# Применение функции к данным
data['errors'] = data['name'].apply(lambda x: generate_errors(x, 5))

In [11]:
data = data.explode('errors').reset_index(drop=True)
data['errors'] = data['errors'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [12]:
data = data[['name', 'errors']]

In [13]:
data

,name,errors
0,Авангард Московская область,Авангард Москоаская область
1,Авангард Московская область,Аватгард Московсшая отласть
2,Авангард Московская область,Аваншард Московсшая ебласть
3,Авангард Московская область,Авашгард Московская еблвсть
4,Авангард Московская область,Аванрард Моиковская область
...,...,...
1525,"ООО ""Триумф"" Москва","ООО ""Трнумф"" Москва"
1526,"ООО ""Триумф"" Москва","ООО ""Тригмф"" Момква"
1527,"ООО ""Триумф"" Москва","ООО ""Тршумр"" Москва"
1528,"ООО ""Триумф"" Москва","ООО ""Триумф"" Москвк"


In [14]:
train, test = train_test_split(data, test_size=0.35, random_state=12345)

In [15]:
train.shape

(994, 2)

In [16]:
test.shape

(536, 2)

In [17]:
model = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
data

,name,errors
0,Авангард Московская область,Авангард Москоаская область
1,Авангард Московская область,Аватгард Московсшая отласть
2,Авангард Московская область,Аваншард Московсшая ебласть
3,Авангард Московская область,Авашгард Московская еблвсть
4,Авангард Московская область,Аванрард Моиковская область
...,...,...
1525,"ООО ""Триумф"" Москва","ООО ""Трнумф"" Москва"
1526,"ООО ""Триумф"" Москва","ООО ""Тригмф"" Момква"
1527,"ООО ""Триумф"" Москва","ООО ""Тршумр"" Москва"
1528,"ООО ""Триумф"" Москва","ООО ""Триумф"" Москвк"


In [19]:
corpus = model.encode(data.name.values)

In [20]:
corpus

array([[-0.01672535,  0.02174135,  0.0296597 , ..., -0.05356811,
        -0.03378502, -0.0686802 ],
       [-0.01672535,  0.02174135,  0.0296597 , ..., -0.05356811,
        -0.03378502, -0.0686802 ],
       [-0.01672535,  0.02174135,  0.0296597 , ..., -0.05356811,
        -0.03378502, -0.0686802 ],
       ...,
       [-0.01976964, -0.02988719, -0.02051289, ..., -0.06708463,
        -0.01758503, -0.00791052],
       [-0.01976964, -0.02988719, -0.02051289, ..., -0.06708463,
        -0.01758503, -0.00791052],
       [-0.01976964, -0.02988719, -0.02051289, ..., -0.06708463,
        -0.01758503, -0.00791052]], dtype=float32)

In [21]:
test

,name,errors
677,ПроСинхро Москва,ПриСкнхре Моснвп
1497,"ООО ""СетПоинт"" Москва","ООО ""СртПиинт"" Москва"
927,СШ № 6 Мурманская область,СШ № 6 Мурманская область
131,Буревестник Санкт-Петербург,Буревестник Санкт-Петекбукг
298,Ермак Иркутская область,Ермак Иркутская область
...,...,...
901,СШ № 1 Вологодская область,СШ № 1 Валогадсная область
673,Прибой Тюменская область,Пргбоц Тюменсквя облпсть
133,Буревестник Санкт-Петербург,Буреаемтшик Саннт-Пеьербург
558,НЛФК Свердловская область,НЛФК Свердловсшая областс


In [22]:
query = model.encode(test.errors.values)

In [23]:
query.shape

(536, 768)

In [24]:
test.shape

(536, 2)

In [25]:
result_id = util.semantic_search(query, corpus, top_k=1)

In [26]:
test['candidate_idx'] = [x[0]['corpus_id'] for x in result_id]

In [27]:
top = [x[0]['corpus_id'] for x in result_id]

In [28]:
test['candidate_name'] = data.name.values[test.candidate_idx.values]

In [29]:
test

,name,errors,candidate_idx,candidate_name
677,ПроСинхро Москва,ПриСкнхре Моснвп,675,ПроСинхро Москва
1497,"ООО ""СетПоинт"" Москва","ООО ""СртПиинт"" Москва",1495,"ООО ""СетПоинт"" Москва"
927,СШ № 6 Мурманская область,СШ № 6 Мурманская область,925,СШ № 6 Мурманская область
131,Буревестник Санкт-Петербург,Буревестник Санкт-Петекбукг,130,Буревестник Санкт-Петербург
298,Ермак Иркутская область,Ермак Иркутская область,297,Ермак Иркутская область
...,...,...,...,...
901,СШ № 1 Вологодская область,СШ № 1 Валогадсная область,900,СШ № 1 Вологодская область
673,Прибой Тюменская область,Пргбоц Тюменсквя облпсть,670,Прибой Тюменская область
133,Буревестник Санкт-Петербург,Буреаемтшик Саннт-Пеьербург,130,Буревестник Санкт-Петербург
558,НЛФК Свердловская область,НЛФК Свердловсшая областс,555,НЛФК Свердловская область


In [30]:
(test.name == test.candidate_name).sum()/test.shape[0]

0.9402985074626866

In [31]:
(test.name == test.candidate_name).sum()

504

In [32]:
test.shape

(536, 4)

In [33]:
(test.name != test.candidate_name).sum()

32

In [34]:
train

,name,errors
989,СШОР по фигурному катанию на коньках Санкт-Пет...,СШОР по фигурному катанию на коньках Санкт-Пет...
696,РОО СФФК РСЯ Республика Саха,РОО СФФК РСЯ Республика Сафп
368,Конаковский лед Тверская область,Кошаковский лед Тверская область
501,Медведково Москва,Мгжвежково Москва
134,Буревестник Санкт-Петербург,Буревестнин Санкт-Петерьуро
...,...,...
1444,АНО ШФК Республика Башкортостан,АНО ШФК Республика Башкортасташ
129,Брянск Брянская область,Брснск Брянская обдасть
1309,Юность Мурманская область,Юшость Мурманская отласть
1509,Прогресс Алтайский край,Прогкесс Алтайскнй край


In [35]:
make_example = lambda x: InputExample(texts=[x['name'], x['errors']])

In [36]:
examples = train[['name', 'errors']].apply(make_example, axis=1).values

In [37]:
train_dataloader = DataLoader(examples, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [38]:
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

In [39]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

def my_f(test_titles, corpus, top_k=1, model = None):
    if model is None:
        model = SentenceTransformer('sentence-transformers/LaBSE')

    query_emb = model.encode(test_titles)
    corpus_emb = model.encode(corpus)

    hits = util.semantic_search(query_emb, corpus_emb, top_k=top_k)

    top = [x[0]['corpus_id'] for x in hits]

    return top

In [40]:
test_titles = ["Афагарден"]
corpus = data.name.values

top_k_schools = my_f(test_titles, corpus, top_k=1)
print(f'Top k schools for "{test_titles[0]}" are {top_k_schools[0]}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Top k schools for "Афагарден" are 0


In [41]:
data

,name,errors
0,Авангард Московская область,Авангард Москоаская область
1,Авангард Московская область,Аватгард Московсшая отласть
2,Авангард Московская область,Аваншард Московсшая ебласть
3,Авангард Московская область,Авашгард Московская еблвсть
4,Авангард Московская область,Аванрард Моиковская область
...,...,...
1525,"ООО ""Триумф"" Москва","ООО ""Трнумф"" Москва"
1526,"ООО ""Триумф"" Москва","ООО ""Тригмф"" Момква"
1527,"ООО ""Триумф"" Москва","ООО ""Тршумр"" Москва"
1528,"ООО ""Триумф"" Москва","ООО ""Триумф"" Москвк"
